In [4]:
# 导入必要模块
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re
import pandas as pd

# 正则提取格式
findTN = re.compile(r'<div class="team-name c-line-clamp1" data-a-188dee8a="">(.*?)</div>')
findData = re.compile(r'<div data-a-188dee8a="">(.*?)</div>')


def askurl(url):
    # 请求头
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'
    }
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    html = response.read().decode("utf-8")
    return html


def getData(url):
    # 获取网页
    html = askurl(url)
    soup = BeautifulSoup(html, "html.parser")
    itemlist = []
    datalist = []
    teamname = []
    winlist = []
    losslist = []

    for item in soup.find_all('div', class_="team-name c-line-clamp1"):  #
        item = str(item)  # 将每一项转化为字符串，以便正则提取

        tn = re.findall(findTN, item)[0]
        teamname.append(tn)

    # 提取所需数据
    for item in soup.find_all('div', class_="c-span2"):
        item = str(item)
        itemlist.append(item)

    #         wdata = re.findall(findData, item)
    for item in itemlist:
        data = re.findall(findData, item)

        # 将空值去除
        if data:
            datalist.append(''.join(data))
        else:
            pass
    for i in range(5, len(datalist), 4):
        winlist.append(datalist[i])
        losslist.append(datalist[i + 1])

    todataframe(teamname, winlist, losslist)


# 保存
def todataframe(teamname, winlist, losslist):
    data = {}
    data['胜场'] = winlist
    data['负场'] = losslist
    df = pd.DataFrame(data, index=teamname)
    writer = pd.ExcelWriter('../data.xlsx', mode='a', engine='openpyxl')
    df.to_excel(writer, sheet_name='match data')
    writer.save()

if __name__ == '__main__':
    # 网页URL
    url = "https://tiyu.baidu.com/match/lpl/tab/%E6%8E%92%E5%90%8D"
    # 获取数据
    getData(url)

['<div class="c-span2" data-a-0d6f3b8e=""><div class="c-response-img90" data-a-0d6f3b8e=""><div class="c-img c-img-s" data-a-0d6f3b8e=""><img data-a-0d6f3b8e="" src="https://search-operate.cdn.bcebos.com/d58b21cd46c5d0595298131fe8162d87.png"/></div></div></div>', '<div class="c-span2" data-a-6df6b556="" style="text-align:center;max-width:none;"><div data-a-188dee8a="">场次</div></div>', '<div class="c-span2" data-a-6df6b556="" style="text-align:center;max-width:none;"><div data-a-188dee8a="">胜</div></div>', '<div class="c-span2" data-a-6df6b556="" style="text-align:center;max-width:none;"><div data-a-188dee8a="">负</div></div>', '<div class="c-span2" data-a-6df6b556="" style="text-align:center;max-width:none;"><div data-a-188dee8a="">积分</div></div>', '<div class="c-span2" data-a-6df6b556="" style="text-align:center;max-width:none;"><div data-a-188dee8a="">0</div></div>', '<div class="c-span2" data-a-6df6b556="" style="text-align:center;max-width:none;"><div data-a-188dee8a="">0</div></div